## Data exportation

In [964]:
import shutil

for i in range(1, 19):
	shutil.copy2("original_data/PlanetLab/PlanetLabData_" +
		str(i), "data/planetlab/planetlab_data_" + str(i) + ".adj")
for i in range(1, 689):
	shutil.copy2("original_data/Seattle/SeattleData_" +
		str(i), "data/seattle/seattle_data_" + str(i) + ".adj")

## Data transformation: RTT (Round Trip Time)

Measured in seconds $s$.

In [965]:
import pandas as pd
import numpy as np

group = "seattle"
time = 1

In [966]:
source_file = group + "/" + group + "_data_" + str(time)
rtt = pd.read_csv("data/" + source_file + ".adj", sep = "\t", lineterminator = "\n", header = None)
rtt = rtt.to_numpy()

## Network throughput

Measured in megabit per second $Mbit/s$. Extracted randomly from a $\mathcal{N}\left(\mu, \sigma^2\right)$, with $\mu = 70$ and $\sigma = 10$.

In [967]:
np.random.seed(2023)

mean = 70
sd = 10
nt = np.random.normal(mean, sd, size = (rtt.shape[0], rtt.shape[0])).astype(int)

## Availability (1)

Extract randomly from a $\rm{Uniform}\left(0, 1\right)$ and set to 
\begin{equation*}
	\begin{cases}
		1 \text{ (available)} & \text{ if} \geq 0.2 \\
		0 \text{ (unavailable)} & \text{ otherwise}
	\end{cases}.
\end{equation*}

In [968]:
av1 = (np.random.uniform(size = (rtt.shape[0])) >= 0.2) ** 2

## Availability (2)

A router reject the connection from another router (temporary banned).  
Extract randomly from a $\rm{Uniform}\left(0, 1\right)$ and set to 
\begin{equation*}
	\begin{cases}
		1 \text{ (not banned)} & \text{ if} \geq 0.2 \\
		0 \text{ (banned)} & \text{ otherwise}
	\end{cases}.
\end{equation*}

In [969]:
av2 = (np.random.uniform(size = (rtt.shape[0], rtt.shape[0])) >= 0.2) ** 2

## Reliability

Add reliability between the different nodes.  
Set all the relationships as fair: $\frac{1}{2}$.  
Extract randomly from a $\rm{Uniform}\left(-1, 1\right)$ and set to 
\begin{equation*}
	\begin{cases}
		-\frac{3}{4} \text{ (unreliable)} & \text{ if} \geq 0.66 \\
		-\frac{1}{4} \text{ (reliable)} & \text{ if} \leq -0.94
	\end{cases},
\end{equation*}
i.e. the $17\%$ of all the relationships can be considered unreliable and the $3\%$ of them can be considered reliable.

In [970]:
rel = np.ones((rtt.shape[0], rtt.shape[0])) / 2

rand = np.random.uniform(-1, 1, size = (rtt.shape[0], rtt.shape[0]))
for index in range(np.where(rand >= 0.66)[0].shape[0]):
	i = np.where(rand >= 0.66)[0][index]
	j = np.where(rand >= 0.66)[1][index]
	rel[i, j] = 3 / 4
for index in range(np.where(rand <= -0.94)[0].shape[0]):
	i = np.where(rand <= -0.94)[0][index]
	j = np.where(rand <= -0.94)[1][index]
	rel[i, j] = - 1 / 4
for i in range(rtt.shape[0]):
	rel[i, i] = 0
np.where(rand <= -0.96)[0].shape[0]

181

In [971]:
f = open("graphs/rel.g", "w")
f.write(str(rel.shape[0]) + "\n")
for i in range(rel.shape[0]):
	for j in range(rel.shape[0]):
		if i != j:
			f.write(str(i) + " " + str(j) + " " + str(rel[i, j]) + "\n")
f.close()

## Complete adjacency matrix

In [972]:
adj_mat = rtt + 1 / 500 * nt + rel

In [973]:
graph_file = group + "/" + group + "_" + str(time)
f = open("graphs/" + graph_file + ".g", "w")
f.write(str(adj_mat.shape[0]) + "\n")
for i in range(adj_mat.shape[0]):
	for j in range(adj_mat.shape[0]):
		if i != j:
			if (av1[i] != 0 and av1[j] != 0 and av2[i, j] != 0):
				f.write(str(i) + " " + str(j) + " " + str(round(adj_mat[i, j], 4)) + "\n")
f.close()

print(np.max(adj_mat), np.min(adj_mat), np.mean(adj_mat))

10.032 -0.132 0.9331102948678707


## Graph class

In [974]:
class graph:
	def __init__(self):
		self.graph = []
		self.n = 0
	def __add_edge(self, u, v, weight):
		self.graph[u].append((v, weight))
	def print(self):
		for u in range(self.n):
			print(str(u) + " →", end = "")
			for v, weight in self.graph[u]:
				print(" " + str(v) + " (" + str(weight) + ") |", end = "")
			print()
	def load_graph(self, filename):
		with open(filename) as file:
			lines = file.readlines()
		self.n = int(lines[0])
		self.graph = [[] for _ in range(self.n)]
		for i in range(1, len(lines)):
			u, v, weight = lines[i].split()
			self.__add_edge(int(u), int(v), float(weight))

In [975]:
g = graph()
# g.load_graph("graphs/graph_neg.g")
g.load_graph("graphs/" + graph_file + ".g")
# g.load_graph("graphs/rel.g")

## Floyd-Warshall

In [976]:
dist = []
prev = []
path = []

In [977]:
def init_fw():
	dist.clear()
	prev.clear()
	dist.extend([[float("inf")] * g.n for _ in range(g.n)])
	prev.extend([[None] * g.n for _ in range(g.n)])

def floyd_warshall(g):
	init_fw()
	for u in range(g.n):
		for v, weight in g.graph[u]:
			dist[u][v] = weight
			prev[u][v] = u
	for u in range(g.n):
		dist[u][u] = 0
		prev[u][u] = u
	for k in range(g.n):
		for i in range(g.n):
			for j in range(g.n):
				if dist[i][j] > dist[i][k] + dist[k][j]:
					dist[i][j] = dist[i][k] + dist[k][j]
					prev[i][j] = prev[k][j]

def find_path_fw(u, v):
	path.clear()
	if prev[u][v] != None:
		path.insert(0, v)
		while u != v:
			v = prev[u][v]
			path.insert(0, v)

## Bellman-Ford (negative cycle detection)

In [978]:
def init_bf():
	dist.clear()
	prev.clear()
	dist.extend([float("inf")] * g.n)
	prev.extend([None] * g.n)

def bellman_ford(g, source):
	init_bf()
	dist[source] = 0
	flag = False
	for i in range(g.n - 1):
		flag = False
		for u in range(g.n):
			for v, weight in g.graph[u]:
				if dist[u] + weight < dist[v]:
					dist[v] = dist[u] + weight
					prev[v] = u
					flag = True
		if flag == False:
			break
	for u in range(g.n):
		for v, weight in g.graph[u]:
			if dist[u] + weight < dist[v]:
				print("error: negative cycle")
			
def find_path_bf(source, v):
	path.clear()
	if prev[v] != None:
		path.insert(0, v)
		while source != v:
			v = prev[v]
			path.insert(0, v)

## Solution

In [979]:
def path_cost(g, path):
	cost = []
	for i in range(len(path) - 1):
		for j, weight in g.graph[path[i]]:
			if j == (path[i + 1]):
				cost.append(weight)
	return cost

In [980]:
max = -1
max_index = -1
max_source = -1

for source in range(99):
	bellman_ford(g, source)
	for i in range(99):
		find_path_bf(source, i)
		if len(path) > max:
			max = len(path)
			max_index = i
			max_source = source

bellman_ford(g, max_source)
find_path_bf(max_source, max_index)
cost = path_cost(g, path)

print("path: ", path)
print("cost: ", cost, "→", sum(cost))

path:  [15, 7, 67, 91, 52, 4, 81, 13, 16, 87, 56, 71, 53, 20, 98]
cost:  [0.116, -0.016, 0.048, 0.196, 0.054, -0.02, 0.016, -0.054, 0.094, 0.152, 0.036, -0.056, 0.082, 0.234] → 0.882


In [981]:
floyd_warshall(g)
find_path_fw(max_source, max_index)
cost = path_cost(g, path)

print("path: ", path)
print("cost: ", cost, "→", sum(cost))

path:  [15, 7, 67, 91, 52, 4, 81, 13, 16, 87, 56, 71, 53, 20, 98]
cost:  [0.116, -0.016, 0.048, 0.196, 0.054, -0.02, 0.016, -0.054, 0.094, 0.152, 0.036, -0.056, 0.082, 0.234] → 0.882
